# SQL Exam

## Instructions to students

This challenge is designed to determine how much you have learned so far and will test your knowledge on SQL.

The answers for this challenge should be selected on Athena for each corresponding multiple-choice question. The questions are included in this notebook and are numbered according to the Athena questions. The options for each question have also been included.

Do not add or remove cells in this notebook. Do not edit or remove the `%%sql` comment as it is required to run each cell.

**_Good luck!_**

## The TMDb database

In this supplementary exam, you will be exploring [The Movie Database](https://www.themoviedb.org/) – an online movie and TV show database that houses some of the most popular movies and TV shows at your fingertips. The TMDb database supports 39 official languages used in over 180 countries daily and dates back all the way to 2008. 


<img src="https://github.com/Explore-AI/Pictures/blob/master/sql_tmdb.jpg?raw=true" width=80%/>


Below is an Entity Relationship Diagram (ERD) of the TMDb database:

<img src="https://github.com/Explore-AI/Pictures/blob/master/TMDB_ER_diagram.png?raw=true" width=70%/>

As can be seen from the ERD, the TMDb database consists of `12 tables` containing information about movies, cast, genre, and so much more.  

Let's get started!

## Loading the database

Before you begin, you need to prepare your SQL environment.  You can do this by loading the magic command `%load_ext sql`.

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook. 
# If you get an error here, make sure that mysql and pymysql are installed correctly. 

%load_ext sql

Next, go ahead and load your database. To do this, you will need to ensure you have downloaded the `TMDB.db` sqlite file from Athena and have stored it in a known location.

In [2]:
# Establish a connection to the local database using the '%sql' magic command.
# Replace 'password' with our connection password and `db_name` with our database name. 
# If you get an error here, please make sure the database name or password is correct.

%sql sqlite:///TMDB.db

'Connected: @TMDB.db'

If the above line didn't throw out any errors, then you should be good to go. Good luck with the exam! 

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///TMDB.db
Done.


name
actors
casts
genremap
genres
keywordmap
keywords
languagemap
languages
movies
oscars


## Questions on SQL

Use the given cell below each question to execute your SQL queries to find the correct input from the options provided. Your solution should match one of the multiple-choice questions on Athena.

### Question 1

Who won the Oscar for “Actor in a Leading Role” in  2015?

(Hint: The winner is indicated as '1.0'.)

**Options:** 

  - Micheal Fassbender
  - Natalie Portman
  - Leonardo DiCaprio
  - Eddie Redmayne


In [8]:
%%sql
select *
from oscars 
where (year=2015 AND award= "Actor in a Leading Role" AND winner = 1.0);

 * sqlite:///TMDB.db
Done.


year,award,winner,name,film
2015,Actor in a Leading Role,1.0,Leonardo DiCaprio,The Revenant


### Question 2

What query will produce the ten oldest movies in the database?

**Options:**

 - SELECT TOP(10) * FROM movies WHERE release_date ORDER BY release_date ASC

 - SELECT  * FROM movies WHERE release_date IS NOT NULL ORDER BY release_date ASC LIMIT 10

 - SELECT * FROM movies WHERE release_date IS NOT NULL ORDER BY release_date DESC LIMIT 10

 -  SELECT * FROM movies WHERE release_date IS NULL ORDER BY release_date DESC LIMIT 10

In [11]:
%%sql
select min(release_date)
from movies

 * sqlite:///TMDB.db
Done.


min(release_date)
1916-09-04 00:00:00.000000


In [10]:
%%sql
SELECT  movie_id, title, release_date
FROM movies 
WHERE release_date IS NOT NULL ORDER BY release_date ASC LIMIT 10


 * sqlite:///TMDB.db
Done.


movie_id,title,release_date
3059,Intolerance,1916-09-04 00:00:00.000000
3060,The Big Parade,1925-11-05 00:00:00.000000
19,Metropolis,1927-01-10 00:00:00.000000
905,Pandora's Box,1929-01-30 00:00:00.000000
65203,The Broadway Melody,1929-02-08 00:00:00.000000
22301,Hell's Angels,1930-11-15 00:00:00.000000
22649,A Farewell to Arms,1932-12-08 00:00:00.000000
3062,42nd Street,1933-02-02 00:00:00.000000
43595,She Done Him Wrong,1933-02-09 00:00:00.000000
3078,It Happened One Night,1934-02-22 00:00:00.000000


### Question 3

How many unique awards are there in the Oscars table?

**Options:**
 - 141
 - 53 
 - 80
 - 114

In [19]:
%%sql
select count(DISTINCT award)
from Oscars;

 * sqlite:///TMDB.db
Done.


count(DISTINCT award)
114


### Question 4

How many movies are there that contain the word “Spider” within their title?

**Options:**
 - 0
 - 5
 - 1
 - 9

In [24]:
%%sql
select count(movie_id) 
from movies
where title like "%Spider%";


 * sqlite:///TMDB.db
Done.


count(movie_id)
9


### Question 5

How many movies are there that are both in the "Thriller" genre and contain the word “love” anywhere in the keywords?


**Options:**
 - 48
 - 38
 - 14
 - 1

In [32]:
%%sql
with love_movies AS 
    (SELECT m.movie_id, m.title, g.genre_name, k.keyword_name
    FROM movies m
    inner join genremap gm
        on m.movie_id = gm.movie_id
    inner join genres g
        on gm.genre_id = g.genre_id
    inner join keywordmap km
        on m.movie_id = km.movie_id
    inner join keywords k
        on km.keyword_id = k.keyword_id
    where genre_name= "Thriller" AND keyword_name LIKE "love")
select count(movie_id) from love_movies;

 * sqlite:///TMDB.db
Done.


count(movie_id)
14


### Question 6

How many movies are there that were released between 1 August 2006 ('2006-08-01') and 1 October 2009 ('2009-10-01') that have a popularity score of more than 40 and a budget of less than 50 000 000?

 
**Options:**

 - 29
 - 23
 - 28
 - 35

In [37]:
%%sql
with movie_pop As 
    (select movie_id, title, release_date, budget, popularity
    from movies
    where (release_date BETWEEN "2006-08-01" AND "2009-10-01") AND popularity > 40 AND budget < 50000000)
select count(movie_id) from movie_pop;

 * sqlite:///TMDB.db
Done.


count(movie_id)
29


### Question 7

How many unique characters has "Vin Diesel" played so far in the database?

**Options:**
 - 24
 - 19
 - 18
 - 16

In [44]:
%%sql
with actor_name AS
    (select m.movie_id, a.actor_name
    from movies m
    inner join casts c
        on m.movie_id = c.movie_id
    inner join actors a
        on c.actor_id = a.actor_id
    where actor_name= "Vin Diesel")
select count(DISTINCT movie_id) from actor_name;

 * sqlite:///TMDB.db
Done.


count(DISTINCT movie_id)
19


### Question 8

What are the genres of the movie “The Royal Tenenbaums”?


**Options:**
 - Action, Romance
 - Drama, Comedy
 - Crime, Thriller
 - Drama, Romance

In [45]:
%%sql
SELECT m.movie_id, m.title, g.genre_name
    FROM movies m
    inner join genremap gm
        on m.movie_id = gm.movie_id
    inner join genres g
        on gm.genre_id = g.genre_id
where title= "The Royal Tenenbaums";

 * sqlite:///TMDB.db
Done.


movie_id,title,genre_name
9428,The Royal Tenenbaums,Drama
9428,The Royal Tenenbaums,Comedy


### Question 9

What are the three production companies that have the highest movie popularity score on average, as recorded within the database?


**Options:**

 - MCL Films S.A., Turner Pictures, and George Stevens Productions
 - The Donners' Company, Bulletproof Cupid, and Kinberg Genre
 - Bulletproof Cupid, The Donners' Company, and MCL Films S.A
 - B.Sting Entertainment, Illumination Pictures, and Aztec Musique

In [53]:
%%sql
select m.movie_id, avg(m.popularity) as avgpopularity, pc.production_company_name
from movies m
inner join productioncompanymap pcm
    on m.movie_id = pcm.movie_id
inner join productioncompanies pc
    on pcm.production_company_id= pc.production_company_id
group by production_company_name
order by avgpopularity desc
limit 5;

 * sqlite:///TMDB.db
Done.


movie_id,avgpopularity,production_company_name
293660,514.569956,The Donners' Company
118340,481.098624,Bulletproof Cupid
246655,326.92099900000005,Kinberg Genre
20352,234.9200424,Illumination Entertainment
271110,198.372395,Vita-Ray Dutch Productions (III)


### Question 10

How many female actors (i.e. gender = 1) have a name that starts with the letter "N"?


**Options:**

 - 0
 - 355
 - 7335
 - 1949

In [56]:
%%sql
select count(*) from actors
where gender = 1 AND actor_name LIKE "N%";

 * sqlite:///TMDB.db
Done.


count(*)
355


### Question 11

Which genre has, on average, the lowest movie popularity score? 


**Options:**

 - Science Fiction
 - Animation
 - Documentary
 - Foreign

In [60]:
%%sql
select m.movie_id, avg(m.popularity) as avgpopularity, g.genre_name
from movies m
inner join genremap gm
    on m.movie_id = gm.movie_id
inner join genres g
    on gm.genre_id = g.genre_id
group by genre_name
order by avgpopularity asc
limit 5;

 * sqlite:///TMDB.db
Done.


movie_id,avgpopularity,genre_name
7509,0.686786794117647,Foreign
291,3.945723809090908,Documentary
10947,6.389414749999999,TV Movie
16,13.101512475675676,Music
13,15.962426457494416,Romance


### Question 12

Which award category has the highest number of actor nominations (actors can be male or female)? (Hint: `Oscars.name` contains both actors' names and film names.)

**Options:**

- Special Achievement Award
- Actor in a Supporting Role
- Actress in a Supporting Role
- Best Picture



In [68]:
%%sql
select award, count(award) as total 
from oscars
group by award
order by total desc
limit 10;

 * sqlite:///TMDB.db
Done.


award,total
Directing,429
Film Editing,410
Actress in a Supporting Role,400
Actor in a Supporting Role,400
Documentary (Short Subject),348
Documentary (Feature),315
Art Direction,307
Foreign Language Film,300
Cinematography,298
Best Picture,298


### Question 13

For all of the entries in the Oscars table before 1934, the year is stored differently than in all the subsequent years. For example, the year would be saved as “1932/1933” instead of just “1933” (the second indicated year).  Which of the following options would be the appropriate code to update this column to have the format of the year be consistent throughout the entire table (second indicated year only shown)?


**Options:**

- `UPDATE Oscars SET year = RIGHT(year, -4)`
- `UPDATE Oscars SET year = SELECT substr(year, -4)`
- `UPDATE Oscars SET year = substr(year, -4)`
- `UPDATE Oscars year =  substr(year, 4)`

In [ ]:
%%sql
UPDATE Oscars SET year = substr(year, -4)

### Question 14

DStv will be having a special week dedicated to the actor Alan Rickman. Which of the following queries would create a new _view_ that shows the titles, release dates, taglines, and overviews of all movies that Alan Rickman has played in?



**Options:**

- SELECT title, release_date, tagline, overview 
FROM Movies LEFT JOIN Casts ON Casts.movie_id = Movies.movie_id Left JOIN Actors ON Casts.actor_id = Actors.actor_id 
WHERE Actors.actor_name = 'Alan Rickman'
AS VIEW Alan_Rickman_Movies

- CREATE VIEW Alan_Rickman_Movies AS  
SELECT title, release_date, tagline, overview FROM Movies  
LEFT JOIN Casts ON Casts.movie_id = Movies.movie_id Left JOIN Actors
ON Casts.actor_id = Actors.actor_id
WHERE Actors.actor_name = 'Alan Rickman' 


- CREATE NEW VIEW  Name  = Alan_Rickman_Movies AS SELECT title, release_date, tagline, overview FROM Movies LEFT JOIN Casts ON Casts.movie_id = Movies.movie_id Left JOIN Actors ON Casts.actor_id = Actors.actor_id WHERE Actors.actor_name = 'Alan Rickman'

- VIEW Alan_Rickman_Movies AS SELECT title, release_date, tagline, overview FROM Movies LEFT JOIN Casts ON Casts.movie_id = Movies.movie_id Left JOIN Actors ON Casts.actor_id = Actors.actor_id WHERE Actors.actor_name = 'Alan Rickman'

In [70]:
%%sql
CREATE VIEW Alans_Rickman_Movies AS
SELECT title, release_date, tagline, overview FROM Movies
LEFT JOIN Casts 
    ON Casts.movie_id = Movies.movie_id 
Left JOIN Actors 
    ON Casts.actor_id = Actors.actor_id 
WHERE Actors.actor_name = 'Alan Rickman'

 * sqlite:///TMDB.db
Done.


[]

In [71]:
%%sql
select * from Alans_Rickman_Movies
limit 10

 * sqlite:///TMDB.db
Done.


title,release_date,tagline,overview
Love Actually,2003-09-07 00:00:00.000000,The ultimate romantic comedy.,Follows seemingly unrelated people as their lives begin to intertwine while they fall in û and out û of love. Affections languish and develop as Christmas draws near.
Die Hard,1988-07-15 00:00:00.000000,40 Stories. Twelve Terrorists. One Cop.,"NYPD cop, John McClane's plan to reconcile with his estranged wife is thrown for a serious loop when minutes after he arrives at her office, the entire building is overtaken by a group of terrorists. With little help from the LAPD, wisecracking McClane sets out to single-handedly rescue the hostages and bring the bad guys down."
Harry Potter and the Philosopher's Stone,2001-11-16 00:00:00.000000,Let the Magic Begin.,"Harry Potter has lived under the stairs at his aunt and uncle's house his whole life. But on his 11th birthday, he learns he's a powerful wizard -- with a place waiting for him at the Hogwarts School of Witchcraft and Wizardry. As he learns to harness his newfound powers with the help of the school's kindly headmaster, Harry uncovers the truth about his parents' deaths -- and about the villain who's to blame."
Harry Potter and the Chamber of Secrets,2002-11-13 00:00:00.000000,Hogwarts is back in session.,"Ignoring threats to his life, Harry returns to Hogwarts to investigate û aided by Ron and Hermione û a mysterious series of attacks."
Harry Potter and the Prisoner of Azkaban,2004-05-31 00:00:00.000000,Something wicked this way comes.,"Harry, Ron and Hermione return to Hogwarts for another magic-filled year. Harry comes face to face with danger yet again, this time in the form of escaped convict, Sirius Black û and turns to sympathetic Professor Lupin for help."
Harry Potter and the Goblet of Fire,2005-11-05 00:00:00.000000,Dark And Difficult Times Lie Ahead.,"Harry starts his fourth year at Hogwarts, competes in the treacherous Triwizard Tournament and faces the evil Lord Voldemort. Ron and Hermione help Harry manage the pressure û but Voldemort lurks, awaiting his chance to destroy Harry and all that he stands for."
Harry Potter and the Order of the Phoenix,2007-06-28 00:00:00.000000,Evil Must Be Confronted.,"Returning for his fifth year of study at Hogwarts, Harry is stunned to find that his warnings about the return of Lord Voldemort have been ignored. Left with no choice, Harry takes matters into his own hands, training a small group of students û dubbed 'Dumbledore's Army' û to defend themselves against the dark arts."
Harry Potter and the Half-Blood Prince,2009-07-07 00:00:00.000000,Dark Secrets Revealed,"As Harry begins his sixth year at Hogwarts, he discovers an old book marked as 'Property of the Half-Blood Prince', and begins to learn more about Lord Voldemort's dark past."
Galaxy Quest,1999-12-23 00:00:00.000000,A comedy of Galactic Proportions.,"The stars of a 1970s sci-fi show - now scraping a living through re-runs and sci-fi conventions - are beamed aboard an alien spacecraft. Believing the cast's heroic on-screen dramas are historical documents of real-life adventures, the band of aliens turn to the ailing celebrities for help in their quest to overcome the oppressive regime in their solar system."
Perfume: The Story of a Murderer,2006-09-13 00:00:00.000000,Based on the best-selling novel,"Jean-Baptiste Grenouille, born in the stench of 18th century Paris, develops a superior olfactory sense, which he uses to create the world's finest perfumes. However, his work takes a dark turn as he tries to preserve scents in the search for the ultimate perfume."


### Question 15

Which of the statements about database normalisation are true?

**Statements:**
 
i) Database normalisation improves data redundancy, saves on storage space, and fulfils the requirement of records to be uniquely identified.

ii) Database normalisation supports up to the Third Normal Form and removes all data anomalies.

iii) Database normalisation removes inconsistencies that may cause the analysis of our data to be more complicated.

iv) Database normalisation increases data redundancy, saves on storage space, and fulfils the requirement of records to be uniquely identified.

**Options:**

 - (i) and (ii)
 - (i) and (iii)
 - (ii) and (iv)
 - (iii) and (iv)

(i) and (iii)